In [ ]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re 
import time
import urllib
import csv  
import json  
import requests
from csv import writer
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
# Download webdriver from  then unzip and place in existing directory. 

# Credit: reference code MP 2.1 Year 2021 from the teaching team of CS 410 

options = Options()
options.headless = True
driver = webdriver.Chrome('./chromedriver',options=options)

This link has all doctors' name at Cook County Healty at Illinois - https://cookcountyhealth.org/about/physicians-directory/ . There are 823 doctors in total. 

In [ ]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,driver):
    driver.get(url)
    res_html = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

In [ ]:
#extracts all Faculty Profile page urls from the Directory Listing Page
def scrape_dir_page(dir_url,driver):
    print ('-'*20,'Scraping directory page','-'*20)
    faculty_links = []
    faculty_base_url = 'https://cookcountyhealth.org/about/physicians-directory/'
    #execute js on webpage to load faculty listings on webpage and get ready to parse the loaded HTML 
    soup = get_js_soup(dir_url,driver)  
    for link_holder in soup.find_all('h1',class_='elementor-post__title'): #get list of all of class 'name'
        rel_link = link_holder.find('a')['href'] #get url
        #rel_link = rel_link.rsplit('/', 1)[-1]
        print(rel_link)
        #url returned is relative, so we need to add base url
        faculty_links.append(rel_link) 
    print ('-'*20,'Found {} faculty profile urls'.format(len(faculty_links)),'-'*20)
    return faculty_links


In [ ]:
dir_url = 'https://cookcountyhealth.org/about/physicians-directory/' #url of directory listings of CS faculty
list_url = []
faculty_links = []
for i in range(1, 34): 
     url = dir_url + str(i) + '/'
     list_url.append(url)
for j in list_url: 
    fac = scrape_dir_page(j,driver)
    faculty_links.append(fac)
faculty_links

In [ ]:
# Number of physicians in Cook County Health

fac_links = []
for sublist in faculty_links:
    for num in sublist:
        fac_links.append(num)

len(fac_links)

In [ ]:
for url in fac_links: 
    
    # multiple sessions tries to avoid http connections errors https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests/47475019#47475019
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    
    page = session.get(url)


    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('div', class_='blog-post blog-post-wrapper physicians')
    print(results)

    with open('doctors.csv', 'a', newline='', encoding='utf8') as f:
        thewriter = writer(f)
        #header = ['Name', 'Specialty', 'Location', 'Education', 'Residency', 'Board Certification']
        #thewriter.writerow(header)
    
        for result in results: 
            out = result.find('img')['alt']
            name = out.rsplit(', ', 1)[0]
            print(name)
            try: 
                specialty = result.find(text='Specialty:').findNext('h6').text
                specialty = "".join(specialty.rstrip())
                print(specialty)
            except AttributeError:
                specialty = "nil"
                print(specialty)   
            try: 
                location = result.find(text='Location(s):').findNext('h6').text
                location = "".join(location.rstrip())
                location =  re.sub(r"^\W+|\W+$", "", location) #regex reference https://stackoverflow.com/questions/22650506/how-to-rermove-non-alphanumeric-characters-at-the-beginning-or-end-of-a-string
                print(location)
            except AttributeError:
                location = "nil"
                print(location)    
            try: 
                education = result.find(text='Medical Education: ').findNext('h6').text
                education = "".join(education.rstrip())
                print(education)
            except AttributeError:
                education = "nil"
                print(education)
            try: 
                residency = result.find(text='Residency: ').findNext('h6').text
                residency = "".join(residency.rstrip())
                print(residency)
            except AttributeError:
                residency = "nil"
                print(residency)
            try: 
                certification = result.find(text='Board Certification: ').findNext('h6').text
                certification = "".join(certification.rstrip())
                print(certification)
            except AttributeError:
                certification = "nil"
                print(certification)
            doctorinfo = [name, specialty, location, education, residency, certification]
            thewriter.writerow(doctorinfo)

In [ ]:
## CSV to Json Converter Code taken from https://www.codespeedy.com/converting-csv-to-json-array-in-python/

def csv_to_json(csvFile, jsonFile):    
    jsondict = {}  
    with open(csvFile) as csvfile:  
        csv_data = csv.DictReader(csvfile)#Converting the csv data into dictionary
        jsondict["data"]=[]
        for rows in csv_data:  
            print(rows)#Just for reference
            jsondict["data"].append(rows)  #Appending all the csv rows
  
    with open(jsonFile, 'w', encoding='utf8') as jsonfile:  
        #Dumping the data into jsonfile.json
        jsonfile.write(json.dumps(jsondict, indent = 4))  #indent is used for pretty printing 
csvfile = 'doctors-latest.csv'  
jsonfile = 'doctors-test.json'  
  
csv_to_json(csvfile, jsonfile)#Calling the function

In [ ]:
with open('doctors-test.json') as f:
    json_dict = json.load(f)
json_dict

## Function to remove empty keys taken from https://gist.github.com/nlohmann/c899442d8126917946580e7f84bf7ee7

def remove_empty_elements(d):
    """recursively remove empty lists, empty dicts, or None elements from a dictionary"""

    def empty(x):
        return x is None or x == {} or x == [] or x == "nil"

    if not isinstance(d, (dict, list)):
        return d
    elif isinstance(d, list):
        return [v for v in (remove_empty_elements(v) for v in d) if not empty(v)]
    else:
        return {k: v for k, v in ((k, remove_empty_elements(v)) for k, v in d.items()) if not empty(v)}
#remove_empty_elements(json_dict)

import json
with open('doctors-remove-nil.json', 'w') as fp:
    json.dump(remove_empty_elements(json_dict), fp)